# El corrido de Gregorio Cortez

## Mapping the manhunt of Rio Grande border folk hero Gregorio Cortez:  the largest manhunt in U.S. history.

### June 14, 1901 to June 22, 1901

### based on lyrics from the Corrido de Gregorio Cortez (variant x english translation by Americo Paredes)

#### project author Diane Lopez

In [42]:
import voila
import geopandas as gpd

import geograpy
from geograpy import extraction
from geograpy import places
import re
from geograpy.labels import Labels
#from geograpy import geograpy-nltk

import geopy
from geopy.geocoders import Nominatim
import time

import pandas as pd
from shapely.geometry import Point
import shapely
import contextily as ctx
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
#from mordecai import Geoparser
import spacy
from spacy import displacy
import spacy 
from spacy.lang.en import English
from spacy.lang.es import Spanish
import nltk
#from spacy.pipeline import EntityRuler
import json
import random
from spacy.tokens import Doc
from spacy.training import Example
from spacy.language import Language
#! python -m spacy download en_core_web_sm 
nlp = spacy.load("en_core_web_lg")


In [45]:
text = "corrido corpus\ElCorridodeGregorioCortez_X.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_ner_model")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
#print(TxGPE)

In [46]:
spacy.displacy.render(doc, style ='ent')

Geoparsing: Finding places from the corrido lyrics

In [47]:
ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]

listents = ' '.join(map(str, ents))
print(listents)

('Karnes', 19, 25, 'GPE') ('Gonzales', 1052, 1060, 'GPE') ('Belmont', 1141, 1148, 'GPE') ('Laredo', 1551, 1557, 'GPE') ('Encinal', 1865, 1872, 'GPE') ('Tell', 2136, 2140, 'GPE') ('Until', 2837, 2842, 'GPE')


In [55]:
LOC = extraction.Extractor(listents)
LOC.find_geoEntities()

[]

In [56]:
pc = places.PlaceContext(listents)

pc.set_countries()
print (pc.countries) #['United States']

pc.set_regions()
print(pc.regions) #['Texas'])

pc.set_cities()
print(pc.cities) #['Brownsville'])

print(pc.address_strings) #['Brownsville','Texas, United States'])

[]
[]
[]
[]


Laredo, Tejas

In [104]:
geolocator = Nominatim(user_agent="CorridosMap")

In [105]:
laredo=pc.cities[0]
print (laredo)

Cortez


need to know the country code https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Current_codes

In [106]:
laredo_info = geolocator.geocode(laredo)
laredo_lat = (laredo_info.latitude) 
laredo_long= (laredo_info.longitude)
print(laredo_info)
print(laredo_long,',', laredo_lat)

Cortez, Montezuma County, Colorado, 81321, United States
-108.584073 , 37.3494528


Belmont, Tejas/Texas

In [107]:
belmont='Belmont, Gonzales County,Texas'
#belmont=pc.cities[1]
belmont_info=geolocator.geocode(belmont)
belmont_lat = (belmont_info.latitude)
belmont_long= (belmont_info.longitude)
print(belmont_info)
print(belmont_long,',', belmont_lat)

Belmont, Gonzales County, Texas, 78122, United States
-97.6875125 , 29.5241066


Gonzales, Tejas

In [108]:

#gonzales='Gonzales,Texas'
gonzales=pc.cities[2]
gonzales_info=geolocator.geocode(gonzales)
gonzales_lat = (gonzales_info.latitude) 
gonzales_long =(gonzales_info.longitude)
#gonzales=gonzales_info, gonzales_coordinates
print(gonzales_lat, ',', gonzales_long)
print(gonzales_info)

28.8791858 , -97.9062893
Karnes County, Texas, United States


dataframe

In [109]:
greg = pd.DataFrame({'Place':[gonzales, 'Belmont', laredo], 'State': ['Texas', 'Texas', 'Texas'], 
'Latitude':[gonzales_lat, belmont_lat, laredo_lat], 'Longitude': [gonzales_long, belmont_long, laredo_long]})

In [110]:
greg.head()

,Place,State,Latitude,Longitude
0,Karnes,Texas,28.879186,-97.906289
1,Belmont,Texas,29.524107,-97.687512
2,Cortez,Texas,37.349453,-108.584073


Setting Geometry- https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_geometry.html#

Creating a GeoDataFrame from a DataFrame with coordinates https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html

In [111]:
greg_gpd = gpd.GeoDataFrame(
    greg, geometry=gpd.points_from_xy(greg.Longitude, greg.Latitude),  crs="EPSG:4326")

figure out later shapely 2.0 

In [112]:
print(greg_gpd.head())
#greg_gpd.plot()

     Place  State   Latitude   Longitude                     geometry
0   Karnes  Texas  28.879186  -97.906289   POINT (-97.90629 28.87919)
1  Belmont  Texas  29.524107  -97.687512   POINT (-97.68751 29.52411)
2   Cortez  Texas  37.349453 -108.584073  POINT (-108.58407 37.34945)


In [113]:
#greg_gpd_basemap, greg_gpd_extent = ctx.bounds2img(*greg_gpd.total_bounds, zoom=10,   
                                            #source=ctx.providers.OpenStreetMap.Mapnik )

In [114]:
px.set_mapbox_access_token(open("mapboxtoken").read())

fig = px.scatter_mapbox(greg_gpd, 
                        color='Place',
                        animation_group='Place',
                        animation_frame=['Gonzales', 'Belmont', 'Laredo'],
                        lat=greg_gpd.geometry.y, 
                        lon=greg_gpd.geometry.x, 
                        hover_name='Place',
                        size='Latitude',
                        zoom=5,
                        width=500,
                        height=500,
                        title=("Gregorio Cortez on the run (June 14 to 22, 1901)"),
                        mapbox_style='carto-positron'                               
                        )

fig.show()

In [115]:
greg_gpd.to_json()

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"Latitude": 28.8791858, "Longitude": -97.9062893, "Place": "Karnes", "State": "Texas"}, "geometry": {"type": "Point", "coordinates": [-97.9062893, 28.8791858]}}, {"id": "1", "type": "Feature", "properties": {"Latitude": 29.5241066, "Longitude": -97.6875125, "Place": "Belmont", "State": "Texas"}, "geometry": {"type": "Point", "coordinates": [-97.6875125, 29.5241066]}}, {"id": "2", "type": "Feature", "properties": {"Latitude": 37.3494528, "Longitude": -108.584073, "Place": "Cortez", "State": "Texas"}, "geometry": {"type": "Point", "coordinates": [-108.584073, 37.3494528]}}]}'

In [116]:
import os
greg_gpd.to_file(r"C:\Users\dmlpz\Corridos", driver="GeoJSON")

C:\Users\dmlpz\anaconda3\envs\Greg\lib\site-packages\geopandas\io\file.py:299: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



PermissionError: [WinError 5] Access is denied: 'C:\\Users\\dmlpz\\Corridos'

In [ ]:
fig.write_html("../corridos/gregmap.html")